In [ ]:
# !pip install --quiet  datasets # to access squad dataset
# !pip install --quiet pyarrow   # to deal with parquet files for saving dataset if required
# !pip install --quiet  tqdm     # for progress bars
# !pip install --quiet transformers # for t5 model
# !pip install --quiet tokenizers  # tokenizers from HuggingFace
# !pip install --quiet sentencepiece # subword tokenizer used by T5
# !pip install --quiet pytorch-lightning # pytorch wrapper 
# !pip install --quiet torchtext # text utilities

# watch -n 1 free -h
# watch -n 1 nvidia-smi

# torch.cuda.empty_cache() 

# Fetching Datasets

In [ ]:
#imports
import pandas as pd
import torch
from tqdm import tqdm
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from pprint import pprint
import copy
import numpy as np

from collections import defaultdict
import ipdb
import random

import pytorch_lightning as pl
from torch.optim import AdamW
import argparse
from transformers import (
    get_linear_schedule_with_warmup
  )

from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor

from tokenizers import AddedToken

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup,
    AutoTokenizer, 
    LongT5Model,
    LongT5ForConditionalGeneration
) 


# pd.options.display.max_rows , pd.options.display.max_columns  = 100,100  

# device  = 'cuda' if torch.cuda.is_available() else "cpu"
# device

# Check the data 


In [3]:
# df = pd.read_parquet('train_tdm_f1_v1.parquet')#.iloc[:2000,:]
# df = pd.read_parquet('train_squad.parquet')

mode = "tdm"
# mode = "tdms"
train_path = f'../data/train_{mode}_f1_v2_short.parquet' 
validation_path = f'../data/dev_{mode}_f1_v2_short.parquet'

df = pd.read_parquet(f'{train_path}')

df

,template_question,answer
0,Value Prediction Network This paper proposes a...,[{'leaderboard': {'Dataset': 'Atari 2600 Seaqu...
1,Read this and answer the question. If the ques...,[{'leaderboard': {'Dataset': 'Atari 2600 Seaqu...
2,Value Prediction Network This paper proposes a...,[{'leaderboard': {'Dataset': 'Atari 2600 Seaqu...
3,Value Prediction Network This paper proposes a...,[{'leaderboard': {'Dataset': 'Atari 2600 Seaqu...
4,Value Prediction Network This paper proposes a...,[{'leaderboard': {'Dataset': 'Atari 2600 Seaqu...
...,...,...
82675,LogMaster: Mining Event Correlations in Logs o...,"[{'leaderboard': {'Dataset': '', 'Metric': '',..."
82676,LogMaster: Mining Event Correlations in Logs o...,"[{'leaderboard': {'Dataset': '', 'Metric': '',..."
82677,Read this article and answer this question Log...,"[{'leaderboard': {'Dataset': '', 'Metric': '',..."
82678,LogMaster: Mining Event Correlations in Logs o...,"[{'leaderboard': {'Dataset': '', 'Metric': '',..."


In [4]:
str(df.at[5, 'answer'])

"[{'leaderboard': {'Dataset': 'Atari 2600 Seaquest', 'Metric': 'Score', 'Task': 'Atari Games'}}\n {'leaderboard': {'Dataset': 'Atari 2600 Amidar', 'Metric': 'Score', 'Task': 'Atari Games'}}\n {'leaderboard': {'Dataset': 'Atari 2600 Krull', 'Metric': 'Score', 'Task': 'Atari Games'}}\n {'leaderboard': {'Dataset': 'Atari 2600 Alien', 'Metric': 'Score', 'Task': 'Atari Games'}}\n {'leaderboard': {'Dataset': 'Atari 2600 Enduro', 'Metric': 'Score', 'Task': 'Atari Games'}}\n {'leaderboard': {'Dataset': 'Atari 2600 Ms. Pacman', 'Metric': 'Score', 'Task': 'Atari Games'}}\n {'leaderboard': {'Dataset': 'Atari 2600 Crazy Climber', 'Metric': 'Score', 'Task': 'Atari Games'}}\n {'leaderboard': {'Dataset': 'Atari 2600 Q*Bert', 'Metric': 'Score', 'Task': 'Atari Games'}}\n {'leaderboard': {'Dataset': 'Atari 2600 Frostbite', 'Metric': 'Score', 'Task': 'Atari Games'}}]"

In [5]:
# len(str(df.at[7000, 'answer']).split())

In [6]:
# # df = pd.read_parquet('train_tdm_f1_v1.parquet')#.iloc[:2000,:]
# df = pd.read_parquet('train_squad.parquet')

# df

# Creating a Pytorch DataSet for T5 Training and Validation

In [5]:
model_name = ["google/flan-t5", "google/long-t5"]
size = ["-base", "-large", "-xl"]
model_attention = ["","-local", "-tglobal"]


bs = 8
epochs = 5

gpus = 2
# gpus = -1

model_idx = 0
size_idx = 0
model_idx = 0

# model_idx = 1
# size_idx = 0
# model_idx = 1

# # # model_max_length = None
# model_max_length = 6000
# max_len_inp = 5750
# model_max_len_out = 250

model_max_length = 512
max_len_inp = 512
# max_len_inp = 500
# max_len_inp = 512
# model_max_len_out = 130
model_max_len_out = 512


t5_tokenizer = AutoTokenizer.from_pretrained(
    f"{model_name[model_idx]}{model_attention[model_idx]}{size[size_idx]}", model_max_length=model_max_length)
t5_model = T5ForConditionalGeneration.from_pretrained(f"{model_name[model_idx]}{model_attention[model_idx]}{size[size_idx]}")

t5_tokenizer.add_tokens(AddedToken("{", normalized=False))
t5_tokenizer.add_tokens(AddedToken("}", normalized=False))

# t5_model = LongT5Model.from_pretrained(f"{model[model_idx]}{model_attention[model_idx]}{size[size_idx]}")
# t5_model = LongT5ForConditionalGeneration.from_pretrained(f"{model[model_idx]}{model_attention[model_idx]}{size[size_idx]}")

print(f"Max token lenght: {t5_tokenizer.model_max_length}")

Max token lenght: 512


In [6]:
# df["Lenght answer"] = df.answer.apply(lambda x: len(str(x)))
# df["Lenght answer"] = df.answer.apply(lambda x: len(str(x).split()))
# df["Lenght question"] = df.template_question.apply(lambda x: len(x.split()))

# # df["Lenght answer"] = df.answer.apply(lambda x: len(x))
# # df["Lenght question"] = df.template_question.apply(lambda x: len(x))

df["Lenght answer"] = df.answer.apply(lambda x: len(t5_tokenizer.batch_encode_plus(
                [str(x)], 
                truncation = False,
                return_tensors="pt"
            )['input_ids'][0]))
df["Lenght question"] = df.template_question.apply(lambda x: len(t5_tokenizer.batch_encode_plus(
                [str(x)], 
                truncation = False,
                return_tensors="pt"
            )['input_ids'][0]))

df.describe()

Token indices sequence length is longer than the specified maximum sequence length for this model (540 > 512). Running this sequence through the model will result in indexing errors


,Lenght answer,Lenght question
count,82680.000000,82680.000000
mean,239.171081,541.288292
std,473.782091,268.016845
min,46.000000,39.000000
25%,46.000000,335.000000
50%,109.000000,552.000000
75%,253.000000,701.000000
max,14165.000000,3988.000000


In [7]:
class QuestionGenerationDataset(Dataset):
    def __init__(self, tokenizer, filepath, max_len_inp=512,max_len_out=96):
        self.path = filepath

        self.passage_column = "context"
        self.answer = "answer"
        self.question = "question"
        self.template_question = "template_question"

        # self.data = pd.read_csv(self.path)
        self.data = pd.read_parquet(self.path)

        self.max_len_input = max_len_inp
        self.max_len_output = max_len_out
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []
        self._build()

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        src_mask = self.inputs[index]["attention_mask"].squeeze()  #squeeze to get rid of the batch dimension
        target_mask = self.targets[index]["attention_mask"].squeeze()  # convert [batch,dim] to [dim] 

        labels = copy.deepcopy(target_ids)
        labels [labels==0] = -100

        return {
            "source_ids": source_ids, "source_mask": src_mask, 
            "target_ids": target_ids, "target_mask": target_mask,"labels":labels}

    def _build(self):
        for rownum,val in tqdm(self.data.iterrows(), total=len(self.data)): # Iterating over the dataframe
#             passage,answer,target = val[self.passage_column],val[self.answer],val[self.question]
            template_question, answer = val[self.template_question], val[self.answer]

#             input_ = f"context: {passage}  answer: {answer}" # T5 Input format for question answering tasks 
#             target = f"question: {str(target)}" # Output format we require
            
#             input_ = f"context: {passage} question: {str(target)}" # T5 Input format for question answering tasks 
#             target = f"answer: {answer}" # Output format we require
            
#             input_ = f"{str(template_question)} </s>" # T5 Input format for question answering tasks 
#             target = f"{answer} </s>" # Output format we require
            input_ = f"{str(template_question)}" # T5 Input format for question answering tasks 
            target = f"{answer}" # Output format we require
            
            # TODO: Not sure if this is needed as the tokenizer can truncate the output. 
            encoded = t5_tokenizer.batch_encode_plus(
                [input_], 
                truncation = False,
                return_tensors="pt"
            )
            
            if len(encoded['input_ids'][0]) > self.max_len_output:
                continue 
                
#             question_plus = f"answer_me: {str(question)}"
#             question_plus += f" context: {str(context)} </s>"

            # tokenize inputs
            tokenized_inputs = self.tokenizer.batch_encode_plus(
                [input_], 
                max_length = self.max_len_input,
                padding = 'max_length',
                truncation = True,
                return_tensors="pt"
            )
            # tokenize targets
            tokenized_targets = self.tokenizer.batch_encode_plus(
                [target], 
                max_length = self.max_len_output,
#                 padding = 'longest',
                padding = 'max_length',
                truncation = True,
#                 truncation = False,
                return_tensors="pt"
            )

            self.inputs.append(tokenized_inputs)
            self.targets.append(tokenized_targets)

In [8]:
# # train_path = 'train_squad.parquet' # change this accordingly
# # validation_path = 'validation_squad.parquet'

# # train_path = 'train_tdm_f1_v1.parquet' # change this accordingly
# # train_path = 'dev_tdm_f1_v1.parquet' # change this accordingly
# # validation_path = 'dev_tdm_f1_v1.parquet'

# # train_path = 'train_tdm_f1_v1_short.parquet'
# train_path = 'data/train_tdm_f1_v2_short.parquet'
# validation_path = 'data/dev_tdm_f1_v2_short.parquet'


# train_dataset = QuestionGenerationDataset(t5_tokenizer, train_path, 
#                                           max_len_inp=max_len_inp, max_len_out=model_max_len_out)
# # validation_dataset = QuestionGenerationDataset(t5_tokenizer, validation_path, 
# #                                                max_len_inp=max_len_inp, max_len_out=model_max_len_out)

# validation_dataset = train_dataset


train_dataset = QuestionGenerationDataset(t5_tokenizer, train_path, 
                                          max_len_inp = max_len_inp, 
                                          max_len_out = model_max_len_out)
validation_dataset = QuestionGenerationDataset(t5_tokenizer, validation_path, 
                                               max_len_inp = max_len_inp, 
                                               max_len_out = model_max_len_out)


100%|██████████| 23235/23235 [00:42<00:00, 545.75it/s]


In [9]:
# Data Sample

train_sample = train_dataset[100] # thanks to __getitem__
decoded_train_input = t5_tokenizer.decode(train_sample['source_ids'])
decoded_train_output = t5_tokenizer.decode(train_sample['target_ids'])

print(decoded_train_input)
print(decoded_train_output)

Sparse and Dense Data with CNNs: Depth Completion and Semantic Segmentation Convolutional neural networks are designed for dense data, but vision data is often sparse (stereo depth, point clouds, pen stroke, etc.). We present a method to handle sparse depth data with optional dense RGB, and accomplish depth completion and semantic segmentation changing only the last layer. Our proposal efficiently learns sparse features without the need of an additional validity mask. We show how to ensure network robustness to varying input sparsities. Our method even works with densities as low as 0.8% (8 layer lidar), and outperforms all published stateof-the-art on the Kitti depth completion benchmark. To evaluate our proposal we carried out two experimental tasks: depth completion (sec For both tasks we used either sparse depth (sD) Lidar input, dense RGB input, or a fusion of both and tested on both synthetic and real public datasets described in sec Synthia This dataset built with the Unity game

In [10]:
# Data Sample

train_sample = train_dataset[100] # thanks to __getitem__
decoded_train_input = t5_tokenizer.decode(train_sample['source_ids'])
decoded_train_output = t5_tokenizer.decode(train_sample['target_ids'])

print(decoded_train_input)
print(decoded_train_output)

Sparse and Dense Data with CNNs: Depth Completion and Semantic Segmentation Convolutional neural networks are designed for dense data, but vision data is often sparse (stereo depth, point clouds, pen stroke, etc.). We present a method to handle sparse depth data with optional dense RGB, and accomplish depth completion and semantic segmentation changing only the last layer. Our proposal efficiently learns sparse features without the need of an additional validity mask. We show how to ensure network robustness to varying input sparsities. Our method even works with densities as low as 0.8% (8 layer lidar), and outperforms all published stateof-the-art on the Kitti depth completion benchmark. To evaluate our proposal we carried out two experimental tasks: depth completion (sec For both tasks we used either sparse depth (sD) Lidar input, dense RGB input, or a fusion of both and tested on both synthetic and real public datasets described in sec Synthia This dataset built with the Unity game

In [11]:
print(f"Length training {len(train_dataset)}")
print(f"Length validation {len(validation_dataset)}")

Length training 36558
Length validation 4891


In [12]:
len("[{ 'leaderboard': { 'Dataset': 'Human3.6M', 'Metric': 'Average MPJPE (mm)', 'Task</s>")

85

# Fine Tuning T5

In [13]:
import pytorch_lightning as pl
from torch.optim import AdamW
import argparse
from transformers import (
    get_linear_schedule_with_warmup
  )

# from transformers import get_linear_schedule_with_warmup, \
# T5ForConditionalGeneration, AdamW, get_linear_schedule_with_warmup

class T5Tuner(pl.LightningModule):
    # 3e-4
    def __init__(self, t5model, t5tokenizer, batchsize=4):
#     def __init__(self, t5model, t5tokenizer, 
#                  lr=5e-5, num_train_epochs=15, warmup_steps=1000, batchsize=4):
#     super(T5Tuner, self).__init__()    
        super().__init__()
    
        self.model = t5model
        self.tokenizer = t5tokenizer
        self.batch_size = batchsize
#         self.save_hyperparameters()

    def forward(self, input_ids, attention_mask=None, 
                decoder_attention_mask=None, 
                lm_labels=None):
      
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_attention_mask=decoder_attention_mask,
            labels=lm_labels,
        )
        
        return outputs

    def training_step(self, batch, batch_idx):
        outputs = self.forward(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            decoder_attention_mask=batch['target_mask'],
            lm_labels=batch['labels']
        )

        loss = outputs[0]
        self.log('train_loss',loss)
        return loss

    def validation_step(self, batch, batch_idx):
        outputs = self.forward(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            decoder_attention_mask=batch['target_mask'],
            lm_labels=batch['labels']
        )

        loss = outputs[0]
        self.log("val_loss",loss)
        return loss

    def train_dataloader(self):
        return DataLoader(train_dataset, batch_size=self.batch_size,
                          num_workers=2)

    def val_dataloader(self):
        return DataLoader(validation_dataset, 
                          batch_size=self.batch_size,
                          num_workers=2)

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=3e-4, eps=1e-8)
        return optimizer
        
#     def configure_optimizers(self):
#         # create optimizer
#         optimizer = AdamW(self.parameters(), lr=self.hparams.lr)
#         # create learning rate scheduler
#         num_train_optimization_steps = self.hparams.num_train_epochs * len(\
# self.train_dataloader())
#         lr_scheduler = {
# 'scheduler': get_linear_schedule_with_warmup(optimizer,
#                         num_warmup_steps=self.hparams.warmup_steps,
#                         num_training_steps=num_train_optimization_steps),
#                         'name': 'learning_rate',
#                         'interval':'step',
#                         'frequency': 1}
        
#         return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}
    


In [14]:
# device

In [15]:
gpus

2

In [16]:
early_stop_callback = EarlyStopping(
   monitor='val_loss',
   min_delta=0.00,
   patience=5,
   strict=False,
   verbose=False,
   mode='min'
)

# for early stopping, 
# see https://pytorch-lightning.readthedocs.io/en/1.0.0/early_stopping\
# .html?highlight=early%20stopping
# early_stop_callback = EarlyStopping(
#     monitor='validation_loss',
#     patience=3,
#     strict=False,
#     verbose=False,
#     mode='min'
# )

# lr_monitor = LearningRateMonitor(logging_interval='step')

model = T5Tuner(t5_model, t5_tokenizer, batchsize=bs)

trainer = pl.Trainer(max_epochs = epochs,
#                      gpus=gpus,
                     strategy='ddp',
                     accelerator='cuda', 
                     devices=gpus,
                     callbacks=[
                         early_stop_callback, 
#                          lr_monitor
                     ]
                    )

trainer.fit(model)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/nfs/home/kabenamualus/.local/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:68: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  "Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning`"
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/1
----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distribu

Sanity Checking: 0it [00:00, ?it/s]

/nfs/home/kabenamualus/.local/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:229: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
/nfs/home/kabenamualus/.local/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:543: PossibleUserWarning: It is recommended to use `self.log('val_loss', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
  category=PossibleUserWarning,
/nfs/home/kabenamualus/.local/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:229: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider 

Training: 0it [00:00, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 32.00 MiB (GPU 0; 23.69 GiB total capacity; 22.22 GiB already allocated; 4.06 MiB free; 22.50 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [20]:
# torch.cuda.empty_cache() 

In [21]:
# # saving the model
# !mkdir "t5_tokenizer"
# !mkdir "t5_trained_model"
# model.model.save_pretrained('t5_trained_model_QA')
# model.model.save_pretrained('t5_trained_model_QA__')

# t5_tokenizer.save_pretrained('t5_tokenizer_QA')

In [20]:
# model[model_idx].replace("/","_").replace("-","_")

In [24]:
# model.model.save_pretrained(\
#f'model_ckpt/{model[model_idx].replace("/","_").replace("-","_")}{model_attention[model_idx].replace("-","_")}{size[size_idx].replace("-","_")}_{bs}_{epochs}_{model_max_length}_{max_len_inp}_{model_max_len_out}')
#f'model_ckpt/{model_name[model_idx].replace("/","_").replace("-","_")}{model_attention[model_idx].replace("-","_")}{size[size_idx].replace("-","_")}_{bs}_{epochs}_{model_max_length}_{max_len_inp}_{model_max_len_out}'

'model_ckpt/google_flan_t5_large_3_5_512_400_112'

# Inference / Predictions

In [13]:
!pwd

/nfs/home/kabenamualus/Research/T5-Leaderboard-QA


In [16]:
# trained_model_path = f'Longt5_trained_tdms_model_QA_bs_{bs}_epochs_{epochs}_model_max_length_{model_max_length}_max_len_inp_{max_len_inp}_model_max_len_out_{model_max_len_out}'
# trained_tokenizer = f'Longt5_tokenizer_tdms_QA_bs_{bs}_epochs_{epochs}_model_max_length_{model_max_length}_max_len_inp_{max_len_inp}_model_max_len_out_{model_max_len_out}'

# trained_model_path = f'./Longt5_trained_tdms_{bs}_{epochs}_{model_max_length}_{max_len_inp}_{model_max_len_out}'
# trained_tokenizer = f'./Longt5_tokenizer_tdms_{bs}_{epochs}_{model_max_length}_{max_len_inp}_{model_max_len_out}'

trained_model_path = f'/nfs/home/kabenamualus/Research/T5-Leaderboard-QA/model_ckpt/tdm_google_flan_t5_large_2_5_512_432_80'
trained_tokenizer = f'/nfs/home/kabenamualus/Research/T5-Leaderboard-QA/model_ckpt/tdm_google_flan_t5_large_tokenizer_2_5_512_432_80'

# trained_model_path = f'./Longt5_trained_tdms_6_5_512_400_112'
# trained_tokenizer = f'./Longt5_tokenizer_tdms_6_5_512_400_112'

# device  = 'cuda' if torch.cuda.is_available() else "cpu"
device = "cpu"

In [17]:
model = T5ForConditionalGeneration.from_pretrained(trained_model_path)
tokenizer = T5Tokenizer.from_pretrained(trained_tokenizer)

In [22]:
# context ="President Donald Trump said and predicted that some states would reopen this month."
# # answer = "Donald Trump"
# question = "Who is the pre"
# text = "context: "+context + " " + "answer: " + answer
# print(text)

# Data Sample Validation 

dev_sample = validation_dataset[500] # thanks to __getitem__
decoded_dev_input = t5_tokenizer.decode(dev_sample['source_ids'])
decoded_dev_output = t5_tokenizer.decode(dev_sample['target_ids'])

print(decoded_dev_input)
print(decoded_dev_output)

Learning Event-Based Motion Deblurring Recovering sharp video sequence from a motion-blurred image is highly ill-posed due to the significant loss of motion information in the blurring process. For event-based cameras, however, fast motion can be captured as events at high time rate, raising new opportunities to exploring effective solutions. In this paper, we start from a sequential formulation of event-based motion deblurring, then show how its optimization can be unfolded with a novel end-to-end deep architecture. The proposed architecture is a convolutional recurrent neural network that integrates visual and temporal knowledge of both global and local scales in principled manner. To further improve the reconstruction, we propose a differentiable directional event filtering module to effectively extract rich boundary prior from the stream of events. We conduct extensive experiments on the synthetic GoPro dataset and a large newly introduced dataset captured by a DAVIS240C camera. Th

In [23]:
# context ="Since its topping out in 2013, One World Trade Center in New York City has been the tallest skyscraper in the United States."
# # answer = "World Trade Center"
# # question = "When did the One World Trade Center became the tallest skyscraper"
# question = "Where is the world trade center ?"
# text = "context: "+context + " " + "answer: " + question
# # text = "context: "+context + " " + "answer: " + answer

# print(text)

In [27]:
encoding = tokenizer.encode_plus(decoded_dev_input,max_length =512,padding='max_length', 
                                 truncation = True,
                                 return_tensors="pt")#.to(device)
print (encoding.keys())
# input_ids,attention_mask  = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)
input_ids,attention_mask  = encoding["input_ids"], encoding["attention_mask"]

dict_keys(['input_ids', 'attention_mask'])


In [25]:
model.eval()
beam_outputs = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_length=model_max_len_out, # How long the generated questions should be
    early_stopping=True,
    num_beams=5,
    num_return_sequences=5
)

for beam_output in beam_outputs:
    sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    print(sent)

['leaderboard': 'Dataset': '', 'Metric': '', 'Task': '']
['leaderboard': 'Dataset': '', 'Metric': '', 'Task': '']  (
['leaderboard': 'Dataset': '', 'Metric': '', 'Task': '']  2015
['leaderboard': 'Dataset': '', 'Metric':'(', 'Task': '']
['leaderboard': 'Dataset': '', 'Metric': '', 'Task': ''] '
